<a href="https://colab.research.google.com/github/Jiablero/Disasters/blob/master/Lusitania_season1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Подготовка данных о пасажирах и команде "Луизитании", находившихся на борту во время гибели судна. 

Данные взяты отсюда: https://www.rmslusitania.info/multimedia/downloads/

In [80]:
import pandas as pd
import numpy as np

In [81]:
data = pd.read_excel('/content/drive/My Drive/disasters/lui_completemanifest.xls',  index_col = False)

In [82]:
data.groupby('Fate').aggregate({'Fate': 'count'})

,Fate
Fate,
Lost,1193
Not on board,1
Saved,763
Saved (died from trauma),4


In [83]:
data = data.drop(['Citizenship', 'Status', 'City', 'County', 'State', 'Country', 'Lifeboat', 'Rescue Vessel', 'Body No.', 'Cabin No.', 'Traveling Companions and other notes', 'Ticket No.', 'Value', 'Adult/Minor'], axis=1)

In [84]:
data.isnull().sum()

Family name            0
Title                  0
Personal name          3
Fate                   0
Age                  653
Department/Class       0
Passenger/Crew         0
Position            1270
Sex                    0
dtype: int64

In [85]:
data.Position[(data.Position.isna()) & (data['Passenger/Crew'] == 'Crew')] = 'Crew'
# остальные с Position = NAN - пассажиры
data.Position = data.Position.fillna('Passenger')

In [86]:
data.Age = data.Age.astype(str).str.extract('(-?[0-9]+)').fillna(-1).astype(int)

In [87]:
# Объединяем Имя, Фамилию и еще какое-то имя в одну колонку
data['Name'] = data.Title + ' ' + data['Personal name'] + ' ' + data['Family name']

In [88]:
data = data.drop(['Family name', 'Title', 'Personal name'], axis = 1)

In [89]:
data = data[['Name', 'Position', 'Fate', 'Age','Sex', 'Department/Class']].rename({'Fate': 'Survived'}, axis = 1)

In [90]:
# Приводим Survived в тот же вид как в других датасетах
data.Survived[data.Survived == 'Saved'] = 1
data.Survived[data.Survived != 1 ] = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [91]:
# Смотрим какие позиции есть, чтобы сделать классы
data.groupby('Position').agg({'Position': 'count'}).rename({'Position': 'Count'}, axis=1).reset_index().sort_values(by = 'Count', ascending=False)

,Position,Count
95,Passenger,1268
56,Fireman,145
145,Trimmer,99
62,First Waiter,53
0,Able-Bodied Seaman,42
...,...,...
54,Extra Third Cook,1
53,Extra Third Baker,1
52,Extra Second Cook,1
51,Extra Second Baker,1


In [93]:
data.groupby('Department/Class').agg({'Department/Class': 'count'}).rename({'Department/Class': 'Count'}, axis=1).reset_index().sort_values(by = 'Count', ascending=False)

,Department/Class,Count
4,Second,601
6,Third,370
2,Engineering,313
8,Victualling,306
3,Saloon,291
1,Deck,69
0,Band,5
5,Stowaway,3
7,Third (Distressed British Seaman),3


In [98]:
# классы пассажиров у нас пока не указаны, назовем всех пассажиров 'no Class', а все остальные будут 'Crew'
data['Class'] = np.where(data.Position == 'Passenger', 'no Class', 'Crew')

In [103]:
# смело предполагаем что Saloon - означает 1-й класс
data.Class[data['Department/Class'] == 'Saloon'] = '1st Class'
data.Class[data['Department/Class'] == 'Second'] = '2nd Class'
data.Class[data['Department/Class'] == 'Third'] = '3rd Class'
data.Class[data['Department/Class'] == 'Third (Distressed British Seaman)'] = '3rd Class'


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from 

In [106]:
# попались товарищи без билета! Впрочем, они все равно погибли.
data[data.Class == 'no Class']

,Name,Position,Survived,Age,Sex,Department/Class,Class
1279,NaN,Passenger,0,-1,Male,Stowaway,no Class
1280,NaN,Passenger,0,-1,Male,Stowaway,no Class
1281,NaN,Passenger,0,-1,Male,Stowaway,no Class


In [107]:
data = data[['Name', 'Position', 'Survived', 'Class', 'Age','Sex']]

In [108]:
print('Мужчин:', data.Sex[data.Sex == 'Male'].count(), 'Женщин:', data.Sex[data.Sex == 'Female'].count())

Мужчин: 1443 Женщин: 518


In [109]:
print('Выживших:', data.Survived[data.Survived == 1].count(), \
      'Погибших:', data.Survived[data.Survived == 0].count(), \
      '% выживших:', data.Survived.mean())

Выживших: 763 Погибших: 1198 % выживших: 0.38908720040795514


In [110]:
data.to_csv('/content/drive/My Drive/disasters/lusitania.csv', index=False)

In [111]:
data

,Name,Position,Survived,Class,Age,Sex
0,Mr. Charles W. CAMERON,Crew,0,Crew,38,Male
1,Mr. E. CARR-JONES,Crew,0,Crew,37,Male
2,Mr. Edward DRAKEFORD,Violin,1,Crew,30,Male
3,Mr. Handel HAWKINS,Cello,1,Crew,25,Male
4,Mr. John William HEMINGWAY,Double Bass,1,Crew,27,Male
...,...,...,...,...,...,...
1956,Mr. Alfred WOOD,Saloon Cabin Bed Steward,1,Crew,-1,Male
1957,Mr. Wallace Edkin WOOD,"Barkeeper, Smokeroom Second Cabin",0,Crew,-1,Male
1958,Master (?) Harold Joseph WRIGHT,Steward's Boy,0,Crew,-1,Male
1959,Master George WYNNE,Assistant Cook,1,Crew,16,Male
